# Implmentation of Elliptic Curve Menezes-Qu-Vanstone in Python EC(MQV)

### In this notebook we will explore the implementation of Elliptic Curve Cryptography using the MQV key exchange protocol in Python. 

In [23]:
def add_points(P: tuple, Q: tuple, p: int):
    """
    This function performs point addition on an elliptic curve defined by the 
    equation y^2 ≡ x^3 + ax + b (mod p), where 'a' and 'b' are curve parameters. 
    
    :param P: first point tuple (x, y)
    :param Q: second point tuple (x, y) 
    :param p: mod int value 
    :return: point P + Q tuple (x, y)
    """
    x1, y1 = P
    x2, y2 = Q
     
    if x1 == x2 and y1 == y2:                       # if same point (P == Q)
        beta = (3 * x1 * x2 + a) * pow(2 * y1, -1, p)
    else:                                           # if not same point (P != Q)
        beta = (y2 - y1) * pow(x2 - x1, -1, p)
     
    x3 = (beta * beta - x1 - x2) % p
    y3 = (beta * (x1 - x3) - y1) % p
    
    is_on_curve((x3, y3), p)                        # make sure point P + Q is on curve 
             
    return x3, y3

In [24]:
def is_on_curve(P: tuple, p: int):
    """
    This function Asserts that a point P is on the elliptic 
    curve defined by the equation y^2 ≡ x^3 + ax + b (mod p).

    :param P: point tuple (x, y) to be checked
    :param p: modulus integer value 
    :return: throws an error if P is not on the curve
    """
    x, y = P 
    assert (y * y) % p == (pow(x, 3, p) + (a * x) + b) % p

#### Curve Configuration

This specific curve has the equation y^2 = x^3 + 7. Fixed base point G, modulo p, order n and cofactor h values are mentioned in the following snippet. Note that these values are all public.

In [25]:
# Curve parameters
a = 0
b = 7

# Base point (G) on the elliptic curve
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424)
 
# Modulo (p) for the finite field
p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)
 
# Order (n) of the base point G in the elliptic curve
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

# Co-factor (h) 
h = 1

In [26]:
# check that point G with mod p is on curve 

is_on_curve(G, p)

In [27]:
def apply_double_and_add_method(G: tuple, k: int, p: int):
    """
    The function performs scalar multiplication on an elliptic curve using the Double-and-Add method.
    It efficiently computes the point kG, where G is the base point on the curve.
    
    :param G: base point tuple (x, y) on the elliptic curve
    :param k: scalar multiplier
    :param p: modulus integer value
    :return: result of the scalar multiplication kG, where G is the base point
    """
    target_point = G
    
    k_binary = bin(k)[2:]                             # binary representation of k, excluding the '0b' prefix
     
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]                # using [i: i+1] to split to remove 0b at the beggining
         
        # doubling - always
        target_point = add_points(target_point, target_point, p)
         
        if current_bit == "1":
            target_point = add_points(target_point, G, p)
         
    return target_point

In [28]:
import random

# Alice's private key (random 256 bits ECC key)
ka = random.getrandbits(256)

# Alice's random key (secret)
ra = random.getrandbits(256)

# Alice's public key (Qa = ka * G)
Qa = apply_double_and_add_method(G = G, k = ka, p = p)

# Alice's random point (Ra = ra * G)
Ra = apply_double_and_add_method(G = G, k = ra, p = p)

In [29]:
# Bob's private key (random 256 bits ECC key)
kb = random.getrandbits(256)

# Bob's random key (secret)
rb = random.getrandbits(256)

# Bob's public key (Qb = kb * G)
Qb = apply_double_and_add_method(G = G, k = kb, p = p)

# Bob's random point (Rb = rb * G)
Rb = apply_double_and_add_method(G = G, k = rb, p = p)

In [30]:
import math

# Calculate the value of l
# - Calculate the floor of the logarithm of n to the base 2
# - Add 1 to the result
# - Take the floor of the entire expression
# - Divide the result by 2
# - Take the ceiling of the final result

l = math.ceil((math.floor(math.log(n, 2)) + 1) / 2)

In [31]:
def bar(P: tuple):
    """
    This functions calculates the 'bar' value for a given point on an elliptic curve.

    :param P: Point tuple (x, y) on the elliptic curve
    :return: 'bar' value calculated using the specified formula
    
    """
    x, y = P 
    
    # Calculate the 'bar' value using the given formula:
    # - Take the remainder of x divided by 2^l
    # - Add 2^l to the result
    
    bar_value = ( x % pow(2, l) ) + pow(2, l)
    return bar_value 

In [32]:
# Calculate Alice's signature
# - Compute bar(Ra) using the 'bar' function for Alice's random point Ra
# - Multiply bar(Ra) by Alice's private key ka
# - Add Alice's random key ra to the result
# - Take the modulus of the sum with the order of the base point n

sa = ( ra + bar(Ra) * ka ) % n

In [33]:
# Calculate Bob's signature
# - Compute bar(Rb) using the 'bar' function for Bob's random point Rb
# - Multiply bar(Rb) by Bob's private key kb
# - Add Bob's random key rb to the result
# - Take the modulus of the sum with the order of the base point n

sb = ( rb + bar(Rb) * kb ) % n 

In [34]:
# Key Exchange for Alice
# Ja = h * sa * (Rb + bar(Rb) * Qb)

Ja = apply_double_and_add_method(G = Qb, k = bar(Rb), p = p)
Ja = add_points(P = Ja, Q = Rb, p = p)
Ja = apply_double_and_add_method(G = Ja, k = h * sa, p = p)

In [35]:
# Key Exchange for Bob
# Jb = h * sb * (Ra + bar(Ra) * Qa)

Jb = apply_double_and_add_method(G = Qa, k = bar(Ra), p = p)
Jb = add_points(P = Jb, Q = Ra, p = p)
Jb = apply_double_and_add_method(G = Jb, k = h * sb, p = p)

In [36]:
Ja

(44956923887039319639249868670481387777425871260688806397577501734993500959261,
 7839889847943120217789883504952188756976025208797000152096588149278870855846)

In [37]:
Jb

(44956923887039319639249868670481387777425871260688806397577501734993500959261,
 7839889847943120217789883504952188756976025208797000152096588149278870855846)

In [38]:
def derive_keys(T: tuple):
    """
    This function derives two 128-bit keys (k1 and k2) from a given point on an elliptic curve.
    
    :param T: point tuple (x, y) on the elliptic curve
    :return: two 128-bit keys (k1, k2) derived from the point T
    
    """
    
    tx, ty = T
     
    # Convert x-coordinate of the point to binary
    tx_binary = bin(tx)[2:]
     
    # Crop to 192 bits (considering 192 bits for SHA-256)
    tx_binary_cropped = tx_binary[0:192]
     
    # Restore the cropped binary to an integer
    tx_restored = int(tx_binary_cropped, 2)
     
    # Compute SHA-256 hash of the restored integer
    hash_hex = hashlib.sha256(
        str.encode(str(tx_restored))
    ).hexdigest()
    
    # Convert the hash from hexadecimal to binary
    hash_binary = bin(int(hash_hex, 16))[2:]
     
    # Extract two 128-bit keys from the binary hash
    k1 = int(hash_binary[0:128], 2).to_bytes(16, byteorder="big")
    k2 = int(hash_binary[128:], 2).to_bytes(16, byteorder="big")
     
    return k1, k2

In [39]:
import hashlib

# Bob uses KDF and gets k1, k2 pair
k1b, k2b = derive_keys(Jb)
 
# Alice uses KDF to find k1, k2 pair
k1a, k2a = derive_keys(Ja)

# Assert that the derived key pairs for Alice and Bob are equal
assert k1a == k1b
assert k2a == k2b

In [40]:
def find_mac(message, key):
    """
    This function computes the Message Authentication Code (MAC) using HMAC-SHA256.

    :param message: message for which MAC is calculated
    :param key: key used for HMAC
    :return: Hexadecimal representation of the computed MAC
    
    """
    return hmac.new(
        key, 
        message, 
        hashlib.sha256
    ).hexdigest()

In [41]:
import hmac

# Bob finds MAC for the message with k2 key
# Notice that an attacker does not know k2, so the attacker cannot find tb
msg = f"2BobAlice{Rb[0]}{Rb[1]}{Ra[0]}{Ra[1]}"
tb = find_mac(message = bytes(msg, "utf-8"), key = k2b)
 
# Alice uses k2 to validate tb coming from Bob
msg = f"2BobAlice{Rb[0]}{Rb[1]}{Ra[0]}{Ra[1]}"
t = find_mac(message = bytes(msg, "utf-8"), key = k2a)
assert t == tb
 
# Then Alice finds the mac of the message with k2 key
# Notice that Bob already knows k2, so they can validate ta
msg = f"2AliceBob{Ra[0]}{Ra[1]}{Rb[0]}{Rb[1]}"
ta = find_mac(message = bytes(msg, "utf-8"), key = k2a)
 
# Now Alice sends ta to Bob
 
# Bob verifies ta coming from Alice
msg = f"2AliceBob{Ra[0]}{Ra[1]}{Rb[0]}{Rb[1]}"
t = find_mac(message = bytes(msg, "utf-8"), key = k2b)
assert t == ta

In [42]:
from Crypto.Cipher import AES

# Bob will encrypt a message with k1b
msg = "MATH206 was fun!"

# Create an AES object with k1b in EAX mode
obj_bob = AES.new(k1b, AES.MODE_EAX)

# Encrypt and digest the message
ciphertext, tag = obj_bob.encrypt_and_digest(msg.encode())

# Print the ciphertext
print(f"ciphertext: {ciphertext}")

ciphertext: b'\x8bn:\xf6\x1e\xd6\x9d\xc5\x7f\n\xaf|\xc9\x97\xaf\xd8'


In [43]:
# Alice will decrypt a message with k1a

# Create an AES object for Alice using k1a, EAX mode, and the nonce from Bob's encryption
obj_alice = AES.new(k1a, AES.MODE_EAX, nonce=obj_bob.nonce)

# Decrypt the ciphertext
plaintext = obj_alice.decrypt(ciphertext)

try:
    # Verify the authenticity of the message using the tag
    obj_alice.verify(tag)
    print(f"restored plaintext: {plaintext.decode()}")
except ValueError:
    print("Decryption failed.")

restored plaintext: MATH206 was fun!


# The rest of the code in this notebook is used for testing. You can ignore it.

In [55]:
# x1 = []
# y1 = []

In [56]:
# ### HOW MUCH TIME? COMP EFFIC. FIGURE 3: 

# import time




# # find how many seconds it takes to find 



# # apply add_points method 1000 times 
# n = 1000000

    

# start_time1 = time.time()
    
# for i in range(2,n):
#     init_point = add_points(init_point, G, p)

# seconds_add_method = time.time() - start_time1

# x1.append(n)
# y1.append(seconds_add_method)
    
# #     print(i)



# # start_time2 = time.time()


# # apply_double_and_add_method(G, 100000, p)


# # seconds_double_and_add_method = time.time() - start_time2


# # print(seconds_add_method)
# print(x1)
# print(y1)

In [57]:
# x2 = []
# y2 = []

In [58]:
# n = 1000000

# start_time2 = time.time()

# apply_double_and_add_method(G, n, p)


# seconds_double_and_add_method = time.time() - start_time2

# x2.append(n)
# y2.append(seconds_double_and_add_method)

# print(x2, y2)

In [59]:

# plt.plot(x1, y1, label='add_points_method')
# plt.plot(x2, y2, label='double_and_add_method')
# plt.xlabel('Iteration')
# plt.ylabel('Execution Time (seconds)')
# plt.legend()
# plt.show()




In [112]:
# from cryptography.hazmat.primitives import serialization
# from cryptography.hazmat.backends import default_backend
# from cryptography.hazmat.primitives.asymmetric import ec


# def generate_ecmqv_keys():
#     private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
#     public_key = private_key.public_key()

#     # Serialize private and public keys
#     private_key_bytes = private_key.private_bytes(
#         encoding=serialization.Encoding.PEM,
#         format=serialization.PrivateFormat.PKCS8,
#         encryption_algorithm=serialization.NoEncryption()
#     )

#     public_key_bytes = public_key.public_bytes(
#         encoding=serialization.Encoding.PEM,
#         format=serialization.PublicFormat.SubjectPublicKeyInfo
#     )

#     return private_key_bytes, public_key_bytes


# private_key, public_key = generate_ecmqv_keys()
# print(private_key.decode(), "\n", public_key.decode())

In [113]:
# from cryptography.hazmat.primitives import serialization
# from cryptography.hazmat.primitives.kdf.hkdf import HKDF
# from cryptography.hazmat.primitives import hashes
# from cryptography.fernet import Fernet
# import base64
# from cryptography.hazmat.primitives import padding
# from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes



# def ec_mqv_encryption(private_key_bytes, peer_public_key_bytes, plaintext):
#     # Deserialize the private key
#     private_key = serialization.load_pem_private_key(
#         private_key_bytes,
#         password=None,
#         backend=default_backend()
#     )

#     # Deserialize the public key
#     peer_public_key = serialization.load_pem_public_key(
#         peer_public_key_bytes,
#         backend=default_backend()
#     )

#     # Ensure that both private and public keys use the same elliptic curve
#     if private_key.curve.name != peer_public_key.curve.name:
#         raise ValueError("Elliptic curves of private and public keys do not match")

#     # Derive shared secret from the key exchange
#     shared_secret = private_key.exchange(ec.ECDH(), peer_public_key)

#     # Derive the same encryption key from the shared secret using HKDF
#     derived_key = HKDF(
#         algorithm=hashes.SHA256(),
#         length=32,  # AES-256 key size
#         salt=None,
#         info=b'ec_mqv_encryption',
#         backend=default_backend()
#     ).derive(shared_secret)

#     # Apply padding to the plaintext before encryption
#     padder = padding.PKCS7(algorithms.AES.block_size).padder()
#     plaintext_padded = padder.update(plaintext) + padder.finalize()

#     # Perform encryption using the derived key
#     encryptor = Cipher(algorithms.AES(derived_key), modes.ECB(), backend=default_backend()).encryptor()
#     ciphertext = encryptor.update(plaintext_padded) + encryptor.finalize()

#     return ciphertext


# plaintext = b"My name is Ahmed, a sohomore at Haverford College and this is my final project for MATH206"
# ciphertext = ec_mqv_encryption(private_key, public_key, plaintext)
# print("Ciphertext:", ciphertext)

In [115]:
# from cryptography.hazmat.backends import default_backend
# from cryptography.hazmat.primitives import serialization
# from cryptography.hazmat.primitives.asymmetric import ec
# from cryptography.hazmat.primitives.kdf.hkdf import HKDF
# from cryptography.hazmat.primitives import hashes



# def ec_mqv_decryption(private_key_bytes, peer_public_key_bytes, ciphertext):
#     # Deserialize the private key
#     private_key = serialization.load_pem_private_key(
#         private_key_bytes,
#         password=None,
#         backend=default_backend()
#     )

#     # Deserialize the public key
#     peer_public_key = serialization.load_pem_public_key(
#         peer_public_key_bytes,
#         backend=default_backend()
#     )

#     # Ensure that both private and public keys use the same elliptic curve
#     if private_key.curve.name != peer_public_key.curve.name:
#         raise ValueError("Elliptic curves of private and public keys do not match")

#     # Derive shared secret from the key exchange
#     shared_secret = private_key.exchange(ec.ECDH(), peer_public_key)

#     # Derive the same encryption key from the shared secret using HKDF
#     derived_key = HKDF(
#         algorithm=hashes.SHA256(),
#         length=32,  # AES-256 key size
#         salt=None,
#         info=b'ec_mqv_encryption',
#         backend=default_backend()
#     ).derive(shared_secret)

#     # Perform decryption using the derived key
#     decryptor = Cipher(algorithms.AES(derived_key), modes.ECB(), backend=default_backend()).decryptor()

#     # Decrypt ciphertext
#     plaintext_padded = decryptor.update(ciphertext) + decryptor.finalize()

#     # Remove padding after decryption
#     unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
#     plaintext = unpadder.update(plaintext_padded) + unpadder.finalize()

#     return plaintext



# decrypted_text = ec_mqv_decryption(private_key, public_key, ciphertext)
# print("Decrypted Text:", decrypted_text.decode())